In [1]:
%matplotlib inline

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

In [3]:
import requests
import json
r = requests.get('https://www.niche.com/api/renaissance/results/?type=private&type=public&listURL=best-colleges&page=1&searchType=college')
r = r.text
data = json.loads(r)
#print(json.dumps(data, indent=4))

In [4]:
print(json.dumps(data["entities"], indent=4))
data = data["entities"]

[
    {
        "guid": "1d755237-c671-478d-8020-63cc46eed935",
        "badge": {
            "display": "Best Colleges in America",
            "ordinal": 1,
            "total": 1647,
            "vanityURL": "best-colleges",
            "photoURLs": {
                "desktop": "https://d33a4decm84gsn.cloudfront.net/search/2019/colleges/colleges-best_2048.png",
                "mobile": "https://d33a4decm84gsn.cloudfront.net/search/2019/colleges/colleges-best_640.png"
            }
        },
        "content": {
            "centroid": {
                "lat": 42.360095,
                "lon": -71.092115
            },
            "entity": {
                "genus": "College",
                "isClaimed": false,
                "isPremium": false,
                "name": "Massachusetts Institute of Technology",
                "tagline": [
                    "4 Year",
                    "Cambridge, MA"
                ],
                "type": "College",
                "url":

In [5]:
# recursivejson.py
# https://hackersandslackers.com/extract-data-from-complex-json-python/

def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results


In [6]:
names = extract_values(data, 'name')
urls = extract_values(data, 'url')
for idx, url in enumerate(urls):
    if "https" in url:
        del urls[idx]
    
rankings =  extract_values(data, 'ordinal')
collegeRankings = dict(zip(names, zip(rankings, urls)))
collegeRankings


{'Massachusetts Institute of Technology': (1,
  'massachusetts-institute-of-technology'),
 'Stanford University': (2, 'stanford-university'),
 'Harvard University': (3, 'harvard-university'),
 'Yale University': (4, 'yale-university'),
 'Princeton University': (5, 'princeton-university'),
 'University of Pennsylvania': (6, 'university-of-pennsylvania'),
 'Columbia University': (7, 'columbia-university'),
 'Duke University': (8, 'duke-university'),
 'Brown University': (9, 'brown-university'),
 'California Institute of Technology': (10,
  'california-institute-of-technology'),
 'Washington University in St. Louis': (11,
  'washington-university-in-st-louis'),
 'Rice University': (12, 'rice-university'),
 'University of Notre Dame': (13, 'university-of-notre-dame'),
 'Northwestern University': (14, 'northwestern-university'),
 'University of Chicago': (15, 'university-of-chicago'),
 'Pomona College': (16, 'pomona-college'),
 'Bowdoin College': (17, 'bowdoin-college'),
 'Vanderbilt Univer

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import csv

categories=['African American', 'Asian', 'Hispanic', 'International', 'Multiracial', 'Native American','Pacific Islander', 'Unknown', 'White']
datadict = {}



for name,value in collegeRankings.items():
    link = value[1]
    temparray = []
    r = requests.get("https://www.niche.com/colleges/"+link+"/students/")
    soup = BeautifulSoup(r.text)
    for idx, percentage in enumerate(soup.findAll('div', attrs={'class': 'fact__table__row__value'})):
        if idx > 13:
            temparray.append(percentage.text)
    datadict[name] = temparray

df = pd.DataFrame.from_dict(datadict, orient='index', columns=categories)
df


,African American,Asian,Hispanic,International,Multiracial,Native American,Pacific Islander,Unknown,White
Massachusetts Institute of Technology,6%,25%,15%,12%,6%,0%,0%,2%,34%
Stanford University,6%,20%,14%,14%,10%,1%,0%,1%,34%
Harvard University,6%,20%,11%,8%,5%,0%,0%,4%,44%
Yale University,6%,15%,11%,19%,5%,1%,0%,3%,40%
Princeton University,7%,20%,9%,12%,4%,0%,0%,6%,41%
University of Pennsylvania,7%,20%,9%,13%,4%,0%,0%,4%,43%
Columbia University,7%,17%,13%,16%,5%,0%,0%,3%,37%
Duke University,10%,21%,8%,10%,2%,1%,0%,3%,47%
Brown University,6%,14%,11%,13%,6%,0%,0%,8%,42%
California Institute of Technology,1%,43%,12%,9%,7%,0%,0%,0%,28%
